In [2]:
import pandas as pd

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
##词袋模型

In [6]:
from sklearn.linear_model import LogisticRegression
#回归模型

In [7]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
#过滤警告信息

In [8]:
simplefilter("ignore",category=ConvergenceWarning)

In [16]:
train=pd.read_csv('./datasets/train.csv',encoding_errors='ignore')

In [18]:
train[:10]

,uuid,title,author,abstract,Keywords,label
0,0,Accessible Visual Artworks for Blind and Visua...,"Quero, Luis Cavazos; Bartolome, Jorge Iranzo; ...",Despite the use of tactile graphics and audio ...,accessibility technology; multimodal interacti...,0
1,1,Seizure Detection and Prediction by Parallel M...,"Li, Chenqi; Lammie, Corey; Dong, Xuening; Amir...","During the past two decades, epileptic seizure...",CNN; Seizure Detection; Seizure Prediction; EE...,1
2,2,Fast ScanNet: Fast and Dense Analysis of Multi...,"Lin, Huangjing; Chen, Hao; Graham, Simon; Dou,...",Lymph node metastasis is one of the most impor...,Histopathology image analysis; computational p...,1
3,3,Long-Term Effectiveness of Antiretroviral Ther...,"Huang, Peng; Tan, Jingguang; Ma, Wenzhe; Zheng...",In order to assess the effectiveness of the Ch...,HIV; ART; mortality; observational cohort stud...,0
4,4,Real-Time Facial Affective Computing on Mobile...,"Guo, Yuanyuan; Xia, Yifan; Wang, Jing; Yu, Hui...",Convolutional Neural Networks (CNNs) have beco...,facial affective computing; convolutional neur...,0
5,5,A phenotype-based forward genetic screen ident...,"Yonghe Ding,Di Lang,Jianhua Yan,Haisong Bu,Hon...",Previously we showed the generation of a prote...,Dnajb6; electrocardiogram; genetic diseases; g...,1
6,6,ROS System Facial Emotion Detection Using Mach...,"Martinez, Javier; Vega, Julio",Facial emotion recognition (FER) is a field of...,ROS; low-cost; raspberry Pi; visual attention;...,0
7,7,A Novel Machine Learning Approach for Android ...,"Odat, Esraa; Yaseen, Qussai M.",This paper proposes a machine learning model b...,Malware; Feature extraction; Machine learning ...,0
8,8,Universal blind image quality assessment using...,"Sang, Qingbing; Wu, Xiaojun; Li, Chaofeng; Lu,...",Most current state-of-the-art blind image qual...,completely blind; image quality assessment; si...,0
9,9,EndoNet: A Deep Architecture for Recognition T...,"Twinanda, Andru P.; Shehata, Sherif; Mutter, D...",Surgical workflow recognition has numerous pot...,Laparoscopic videos; cholecystectomy; convolut...,1


In [19]:
train['title']=train['title'].fillna('')
train['abstract'] = train['abstract'].fillna('')

In [20]:
test=pd.read_csv('./datasets/test.csv',encoding_errors='ignore')

In [21]:
test[:5]

,uuid,title,author,abstract,Keywords
0,0,Monitoring Changes in Intracellular Reactive O...,"Al-Hassan M Mustafa,Ramy Ashry,Oliver H Krämer...",Reactive oxygen species (ROS) are induced by s...,Flow cytometry; HDACi; Leukemia; ROS.
1,1,Source Printer Classification Using Printer Sp...,"Joshi, Sharad; Khanna, Nitin",The knowledge of the source printer can help i...,Printer classification; local texture patterns...
2,2,Plasma-processed CoSn/RGO nanocomposite: A low...,"Omelianovych, Oleksii; Larina, Liudmila L.; Oh...",The high cost of state-of-the-art Pt counter e...,Plasma reduction; Bimatalic alloy CoxSn1-x; Re...
3,3,Immediate Antiretroviral Therapy: The Need for...,"Mgbako, Ofole; E. Sobieszczyk, Magdalena; Olen...","Immediate antiretroviral therapy (iART), defin...",HIV; antiretroviral therapy; rapid; health equity
4,4,Design and analysis of an ultra-low-power LC q...,"Lee, Kin Keung; Bryant, Carl; Tormanen, Markus...",This paper presents the design of an ultra-low...,Varactor; Spiral inductor; Quadrature generati...


In [22]:
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')

In [38]:
train['text']=train['title'].fillna('')+' '+train['author'].fillna('')+' '+train['abstract'].fillna('')+' '+train['Keywords'].fillna('')
test['text'] = test['title'].fillna('') + ' ' +  test['author'].fillna('') + ' ' + test['abstract'].fillna('')+ ' ' + train['Keywords'].fillna('')

In [39]:
train['text'][:10],test['text'][:10]

(0    Accessible Visual Artworks for Blind and Visua...
 1    Seizure Detection and Prediction by Parallel M...
 2    Fast ScanNet: Fast and Dense Analysis of Multi...
 3    Long-Term Effectiveness of Antiretroviral Ther...
 4    Real-Time Facial Affective Computing on Mobile...
 5    A phenotype-based forward genetic screen ident...
 6    ROS System Facial Emotion Detection Using Mach...
 7    A Novel Machine Learning Approach for Android ...
 8    Universal blind image quality assessment using...
 9    EndoNet: A Deep Architecture for Recognition T...
 Name: text, dtype: object,
 0    Monitoring Changes in Intracellular Reactive O...
 1    Source Printer Classification Using Printer Sp...
 2    Plasma-processed CoSn/RGO nanocomposite: A low...
 3    Immediate Antiretroviral Therapy: The Need for...
 4    Design and analysis of an ultra-low-power LC q...
 5    A Design Space Exploration Framework for Convo...
 6    ESKAPE Act Plus: Pathway Activation Analysis f...
 7    Three-Dimensio

In [40]:
vector=CountVectorizer().fit(train['text'])
# 从train数据集中获得词表
train_vector=vector.transform(train['text'])
test_vector=vector.transform(test['text'])

In [41]:
train_vector[:5]
#单词表长度67851

<5x67851 sparse matrix of type '<class 'numpy.int64'>'
	with 766 stored elements in Compressed Sparse Row format>

In [43]:
md=LogisticRegression()
md.fit(train_vector,train['label'])
test['label']=md.predict(test_vector)

In [45]:
test['label'][:10]

0    1
1    0
2    0
3    0
4    0
5    0
6    1
7    1
8    1
9    0
Name: label, dtype: int64

In [46]:
test[['uuid','Keywords','label']].to_csv('submit_task1.csv',index=None)